In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing_extensions import List, TypedDict
from langchain_core.prompts import PromptTemplate
from langgraph.graph import START, StateGraph, END
import smtplib
from email.message import EmailMessage


load_dotenv()

openrouter_key = os.getenv("OPENROUTER_API_KEY")
openrouter_base_url = os.getenv("OPENAI_BASE_URL")


llm = ChatOpenAI(
    model_name="deepseek/deepseek-chat-v3.1:free",
    api_key=openrouter_key,
    base_url=openrouter_base_url
)


class State(TypedDict):
    Log_entry: str
    answer: str


def classifier(state: State):
    """
    Classifies a log entry as 'critical' or 'not critical' using an LLM."""

    log_entry = state["Log_entry"]
    if not log_entry:
        return {"classification": "error", "answer": "No log entry provided."}

    template = """You are a log analysis expert. Your task is to classify a log entry as either 'critical' 
            or 'not critical'. Provide a brief, one-sentence reason for your classification.\n\n
            Log Entry: {Log_entry}\n\n
            Classification: """
    

    formatted_prompt = template.format(Log_entry=log_entry)
    response = llm.invoke(formatted_prompt)
    return {"Log_entry": log_entry, "answer": response.content}